In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
sr = 22050 # isti za sve pjesme

data = pd.DataFrame(index=np.array([int(filename[:-4]) for dirname, _, filenames in os.walk('/kaggle/input/fma-small-s-db') for filename in filenames]).astype(np.int32))

def load(song_id):
    return np.load(f'/kaggle/input/fma-small-s-db/S_DB/{song_id}.npy')

data['width'] = [load(track_id).shape[1] for track_id in data.index]

In [3]:
from sklearn.model_selection import train_test_split

height = 128
width = 16
overlap = 0

# Izbacimo pjesme duljine manje od width
data = data[data['width'] >= width]

# Učitajmo podatke o žanrovima
data['genre_name'] = pd.read_csv('/kaggle/input/fma-small/fma_metadata/fma_metadata/tracks.csv', index_col=0, header=[0, 1]).loc[data.index, ('track', 'genre_top')]
genre_encoder = LabelEncoder()
data['genre'] = genre_encoder.fit_transform(data['genre_name']).astype(np.int8)

train_ids, test_ids = train_test_split(data.index, test_size=0.2, stratify=data['genre'], random_state=0)
train_ids, val_ids = train_test_split(train_ids, test_size=0.1, stratify=data.loc[train_ids, 'genre'], random_state=0)

def load_segments(track_ids):
    n_segments = sum(1 + (track_width - width) // (width - overlap) for track_width in data.loc[track_ids, 'width'])
    x = np.empty((n_segments, height, width), dtype=np.float32) # isječci
    y = np.empty((n_segments,), dtype=np.int8)                  # žarn isječka
    z = np.empty((n_segments,), dtype=np.int32)                 # iz koje pjesme isjecak dolazi
    j = 0
    for track_id in track_ids:
        track = load(track_id)
        for end in range(width, track.shape[1], width-overlap):
            x[j] = track[:, end-width:end]
            y[j] = data.loc[track_id, 'genre']
            z[j] = track_id
            j += 1
    x = x.reshape((n_segments, height, width, 1))
    return x, y, z

x_train, y_train, _ = load_segments(train_ids)
x_val, y_val, _ = load_segments(val_ids)
x_test, y_test, z_test = load_segments(test_ids)

In [4]:
minimum = x_train.min()
maximum = x_train.max()
I=maximum-minimum

In [5]:
x_train=(x_train-minimum)/I
x_test = (x_test-minimum)/I
x_val = (x_val-minimum)/I

In [6]:
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, InputLayer, UpSampling2D
from tensorflow.keras.callbacks import TensorBoard
import gc


#convlayers=1 (64)
#convlayers=1 (128)
#convlayers=1 (64) denselayers=1 (16)
#convlayers=1 (64) denselayers=1 (32)
#convlayers=1 (128) denselayers=1 (16)
#convlayers=1 (128) denselayers=1 (32)
#convlayers=2 (32) (64)
#convlayers=2 (32) (64) denselayers=1 (16)
#convlayers=2 (32) (64) denselayers=2 (8) (16)
#convlayers=3 (16) (32) (64)
#convlayers=3 (32) (64) (128) denselayers=1 (16)


modeli = [
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(8, activation="softmax")
    ]),
    
        
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(128, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(8, activation="softmax")
    ]),
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(16, activation="relu"),
        Dense(8, activation="softmax")
    ]),
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(32, activation="relu"),
        Dense(8, activation="softmax")
    ]),
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(128, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(16, activation="relu"),
        Dense(8, activation="softmax")
    ]),
    
        
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(128, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(32, activation="relu"),
        Dense(8, activation="softmax")
    ]),
    
        
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(32, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(8, activation="softmax")
    ]),
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(32, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(16, activation="relu"),
        Dense(8, activation="softmax")
    ]),
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(32, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(8, activation="relu"),
        Dense(16, activation="relu"),
        Dense(8, activation="softmax")
    ]),
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(16, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Conv2D(32, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(8, activation="softmax")
    ]),
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(32, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Conv2D(128, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(16, activation="relu"),
        Dense(8, activation="softmax")
    ])
    
    
]

In [7]:
checkpoints = [tf.keras.callbacks.ModelCheckpoint(
    filepath=f'/kaggle/working/checkpoint{k}',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True) for k in range(1,12)]   
    

In [8]:
from keras import backend as K

sgd = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)


for i in range(11):
    print(f'Treniranje modela broj {i+1}')
    modeli[i].compile(optimizer = sgd,
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'])
    modeli[i].fit(x_train, y_train, validation_data=(x_val, y_val), epochs=70, batch_size=256, callbacks=[checkpoints[i]])
    modeli[i].load_weights(checkpoints[i].filepath)
    modeli[i].save(f'model{i+1}.h5')
    gc.collect()
    print("\n")

Using TensorFlow backend.


Treniranje modela broj 1
Train on 460405 samples, validate on 51200 samples
Epoch 1/70
460405/460405 [==============================] - 17s 37us/sample - loss: 1.8573 - accuracy: 0.3051 - val_loss: 1.7825 - val_accuracy: 0.3529
Epoch 2/70
460405/460405 [==============================] - 16s 36us/sample - loss: 1.7184 - accuracy: 0.3707 - val_loss: 1.7005 - val_accuracy: 0.3953
Epoch 3/70
460405/460405 [==============================] - 17s 36us/sample - loss: 1.6508 - accuracy: 0.4003 - val_loss: 1.6636 - val_accuracy: 0.4038
Epoch 4/70
460405/460405 [==============================] - 17s 37us/sample - loss: 1.6195 - accuracy: 0.4153 - val_loss: 1.6421 - val_accuracy: 0.4178
Epoch 5/70
460405/460405 [==============================] - 16s 36us/sample - loss: 1.5982 - accuracy: 0.4251 - val_loss: 1.6116 - val_accuracy: 0.4310
Epoch 6/70
460405/460405 [==============================] - 16s 36us/sample - loss: 1.5802 - accuracy: 0.4331 - val_loss: 1.5984 - val_accuracy: 0.4381
Epoch 7/70
4

460405/460405 [==============================] - 26s 57us/sample - loss: 1.3352 - accuracy: 0.5330 - val_loss: 1.4666 - val_accuracy: 0.5006
Epoch 38/70
460405/460405 [==============================] - 26s 57us/sample - loss: 1.3310 - accuracy: 0.5349 - val_loss: 1.4880 - val_accuracy: 0.4946
Epoch 39/70
460405/460405 [==============================] - 27s 58us/sample - loss: 1.3273 - accuracy: 0.5368 - val_loss: 1.4756 - val_accuracy: 0.4977
Epoch 40/70
460405/460405 [==============================] - 26s 57us/sample - loss: 1.3235 - accuracy: 0.5380 - val_loss: 1.4594 - val_accuracy: 0.5013
Epoch 41/70
460405/460405 [==============================] - 27s 58us/sample - loss: 1.3200 - accuracy: 0.5389 - val_loss: 1.4591 - val_accuracy: 0.5040
Epoch 42/70
460405/460405 [==============================] - 26s 57us/sample - loss: 1.3146 - accuracy: 0.5411 - val_loss: 1.4707 - val_accuracy: 0.4957
Epoch 43/70
460405/460405 [==============================] - 26s 57us/sample - loss: 1.3123 - 

460405/460405 [==============================] - 17s 36us/sample - loss: 1.6205 - accuracy: 0.4067 - val_loss: 1.6224 - val_accuracy: 0.4166
Epoch 4/70
460405/460405 [==============================] - 17s 37us/sample - loss: 1.5783 - accuracy: 0.4250 - val_loss: 1.5901 - val_accuracy: 0.4399
Epoch 5/70
460405/460405 [==============================] - 17s 37us/sample - loss: 1.5442 - accuracy: 0.4405 - val_loss: 1.5639 - val_accuracy: 0.4500
Epoch 6/70
460405/460405 [==============================] - 17s 38us/sample - loss: 1.5138 - accuracy: 0.4537 - val_loss: 1.5464 - val_accuracy: 0.4625
Epoch 7/70
460405/460405 [==============================] - 17s 36us/sample - loss: 1.4883 - accuracy: 0.4657 - val_loss: 1.5365 - val_accuracy: 0.4621
Epoch 8/70
460405/460405 [==============================] - 17s 37us/sample - loss: 1.4648 - accuracy: 0.4755 - val_loss: 1.5211 - val_accuracy: 0.4732
Epoch 9/70
460405/460405 [==============================] - 17s 38us/sample - loss: 1.4436 - accura

460405/460405 [==============================] - 26s 57us/sample - loss: 1.2051 - accuracy: 0.5804 - val_loss: 1.4532 - val_accuracy: 0.5061
Epoch 40/70
460405/460405 [==============================] - 26s 56us/sample - loss: 1.1982 - accuracy: 0.5827 - val_loss: 1.4634 - val_accuracy: 0.5057
Epoch 41/70
460405/460405 [==============================] - 26s 57us/sample - loss: 1.1906 - accuracy: 0.5855 - val_loss: 1.4671 - val_accuracy: 0.5061
Epoch 42/70
460405/460405 [==============================] - 26s 57us/sample - loss: 1.1825 - accuracy: 0.5885 - val_loss: 1.4567 - val_accuracy: 0.5061
Epoch 43/70
460405/460405 [==============================] - 26s 57us/sample - loss: 1.1775 - accuracy: 0.5905 - val_loss: 1.4712 - val_accuracy: 0.4992
Epoch 44/70
460405/460405 [==============================] - 26s 56us/sample - loss: 1.1692 - accuracy: 0.5930 - val_loss: 1.4631 - val_accuracy: 0.5071
Epoch 45/70
460405/460405 [==============================] - 26s 56us/sample - loss: 1.1596 - 

460405/460405 [==============================] - 24s 53us/sample - loss: 1.5709 - accuracy: 0.4369 - val_loss: 1.5894 - val_accuracy: 0.4405
Epoch 6/70
460405/460405 [==============================] - 25s 54us/sample - loss: 1.5418 - accuracy: 0.4501 - val_loss: 1.5528 - val_accuracy: 0.4617
Epoch 7/70
460405/460405 [==============================] - 24s 53us/sample - loss: 1.5143 - accuracy: 0.4610 - val_loss: 1.5362 - val_accuracy: 0.4708
Epoch 8/70
460405/460405 [==============================] - 24s 53us/sample - loss: 1.4900 - accuracy: 0.4714 - val_loss: 1.5245 - val_accuracy: 0.4747
Epoch 9/70
460405/460405 [==============================] - 25s 54us/sample - loss: 1.4713 - accuracy: 0.4785 - val_loss: 1.5304 - val_accuracy: 0.4679
Epoch 10/70
460405/460405 [==============================] - 24s 53us/sample - loss: 1.4530 - accuracy: 0.4863 - val_loss: 1.4906 - val_accuracy: 0.4898
Epoch 11/70
460405/460405 [==============================] - 25s 54us/sample - loss: 1.4374 - accu

460405/460405 [==============================] - 25s 54us/sample - loss: 1.0747 - accuracy: 0.6271 - val_loss: 1.4879 - val_accuracy: 0.5104
Epoch 42/70
460405/460405 [==============================] - 25s 54us/sample - loss: 1.0675 - accuracy: 0.6295 - val_loss: 1.5122 - val_accuracy: 0.4999
Epoch 43/70
460405/460405 [==============================] - 25s 54us/sample - loss: 1.0615 - accuracy: 0.6313 - val_loss: 1.5104 - val_accuracy: 0.5074
Epoch 44/70
460405/460405 [==============================] - 25s 54us/sample - loss: 1.0544 - accuracy: 0.6339 - val_loss: 1.5539 - val_accuracy: 0.4980
Epoch 45/70
460405/460405 [==============================] - 25s 54us/sample - loss: 1.0470 - accuracy: 0.6367 - val_loss: 1.5130 - val_accuracy: 0.5033
Epoch 46/70
460405/460405 [==============================] - 25s 54us/sample - loss: 1.0418 - accuracy: 0.6387 - val_loss: 1.5194 - val_accuracy: 0.5009
Epoch 47/70
460405/460405 [==============================] - 25s 53us/sample - loss: 1.0372 - 

460405/460405 [==============================] - 21s 46us/sample - loss: 1.5039 - accuracy: 0.4635 - val_loss: 1.5246 - val_accuracy: 0.4671
Epoch 8/70
460405/460405 [==============================] - 21s 45us/sample - loss: 1.4816 - accuracy: 0.4730 - val_loss: 1.5434 - val_accuracy: 0.4530
Epoch 9/70
460405/460405 [==============================] - 21s 45us/sample - loss: 1.4601 - accuracy: 0.4818 - val_loss: 1.4965 - val_accuracy: 0.4850
Epoch 10/70
460405/460405 [==============================] - 21s 47us/sample - loss: 1.4415 - accuracy: 0.4881 - val_loss: 1.5022 - val_accuracy: 0.4843
Epoch 11/70
460405/460405 [==============================] - 21s 45us/sample - loss: 1.4250 - accuracy: 0.4941 - val_loss: 1.4963 - val_accuracy: 0.4873
Epoch 12/70
460405/460405 [==============================] - 21s 45us/sample - loss: 1.4070 - accuracy: 0.5017 - val_loss: 1.4752 - val_accuracy: 0.4941
Epoch 13/70
460405/460405 [==============================] - 21s 46us/sample - loss: 1.3920 - ac

460405/460405 [==============================] - 38s 84us/sample - loss: 0.8079 - accuracy: 0.7219 - val_loss: 1.7028 - val_accuracy: 0.4931
Epoch 44/70
460405/460405 [==============================] - 39s 84us/sample - loss: 0.7969 - accuracy: 0.7256 - val_loss: 1.6696 - val_accuracy: 0.4944
Epoch 45/70
460405/460405 [==============================] - 38s 84us/sample - loss: 0.7877 - accuracy: 0.7293 - val_loss: 1.7003 - val_accuracy: 0.4931
Epoch 46/70
460405/460405 [==============================] - 39s 84us/sample - loss: 0.7764 - accuracy: 0.7333 - val_loss: 1.7051 - val_accuracy: 0.5005
Epoch 47/70
460405/460405 [==============================] - 38s 83us/sample - loss: 0.7662 - accuracy: 0.7369 - val_loss: 1.7006 - val_accuracy: 0.4971
Epoch 48/70
460405/460405 [==============================] - 39s 84us/sample - loss: 0.7589 - accuracy: 0.7393 - val_loss: 1.7372 - val_accuracy: 0.4800
Epoch 49/70
460405/460405 [==============================] - 39s 84us/sample - loss: 0.7499 - 

In [9]:
#maksimalni val_acc za pojedini model

for i in range(11):
    print(modeli[i].evaluate(x_val,y_val)[1])

0.50683594
0.511875
0.51310545
0.50857425
0.5137109
0.51625
0.51027346
0.5111523
0.5101563
0.5116797
0.5144922


In [10]:
best_model_number = 0
max_val_accuracy = modeli[0].evaluate(x_val,y_val)[1]

for i in range(1,11):
    temp_val_accuracy = modeli[i].evaluate(x_val,y_val)[1]
    
    if max_val_accuracy < temp_val_accuracy:
        best_model_number = i
        max_val_accuracy = temp_val_accuracy


In [11]:
from sklearn.metrics import accuracy_score


pred = pd.DataFrame(modeli[best_model_number].predict(x_test), index=z_test)
# Svaki žanr glasa za isječak s maksimalnom vjerojatnosti, zatim gledamo za koji žanr najviše isječaka glasa
votes = pred.idxmax(axis=1)
score=accuracy_score(data.loc[test_ids, 'genre'].sort_index(), votes.groupby(votes.index).agg(lambda x:x.value_counts().index[0]))

In [12]:
print(f"točnost na test skupu je:{score}")

točnost na test skupu je:0.586875
